In [1]:
# Installing Libraries
# !pip install tflearn

     -------------------------------------- 107.3/107.3 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=552f166efcd527664350e35956c6cf75f3ae73e1529417f22101302acbf51374
  Stored in directory: c:\users\dzope\appdata\local\pip\cache\wheels\4a\d5\f8\9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
# pip install tflearn

  Using cached tflearn-0.5.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dzope\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
#Imports
import nltk
nltk.download('punkt')
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dzope\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
#Loading the intents data
with open("intents.json") as file:
	data = json.load(file)

In [6]:
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

In [7]:
#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    #Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])

In [8]:
#Pre-Processing the given data
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [9]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

In [11]:
#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)

In [12]:
#Generating and Saving ML Model
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 799  | total loss: 0.82671 | time: 0.014s
| Adam | epoch: 200 | loss: 0.82671 - acc: 0.6208 -- iter: 24/25
Training Step: 800  | total loss: 0.77989 | time: 0.019s
| Adam | epoch: 200 | loss: 0.77989 - acc: 0.6588 -- iter: 25/25
--
INFO:tensorflow:d:\Alwi\TUGAS KULIAH SEMESTER 8\chatbot_yelahwi\ML\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
